In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np

### Load data

In [2]:
DatName='DYAD06NF_'
DatRaw=pd.read_excel('/Users/ruzenkakaldenbach/Desktop/Drive/raw_data/'+DatName+'.xlsx')
DatRaw

,Unnamed: 0,date,err,frame_count,frame_number,frame_timestamp,name,oid,type,x,y
0,0,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_center,1,BVIEW_annotated_object_2d,729.542549,513.611408
1,1,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_front,2,BVIEW_annotated_object_2d,743.034555,539.561709
2,2,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_hat,3,BVIEW_annotated_object_2d,735.060775,526.115079
3,3,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,red_center,5,BVIEW_annotated_object_2d,764.611729,880.072303
4,4,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,red_front,6,BVIEW_annotated_object_2d,761.883141,874.760670
...,...,...,...,...,...,...,...,...,...,...,...
308425,971,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,red_front,6,BVIEW_annotated_object_2d,1051.591755,270.523982
308426,972,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,red_hat,7,BVIEW_annotated_object_2d,1051.443252,256.990785
308427,973,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,yellow_center,9,BVIEW_annotated_object_2d,1230.350601,885.085590
308428,974,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,yellow_front,10,BVIEW_annotated_object_2d,1214.486095,825.492977


### Create empty dataframe

In [3]:
DF = pd.DataFrame(columns=['xrc', 'yrc', 'xrf', 'yrf', 
                           'xbc', 'ybc', 'xbf', 'ybf', 
                           'xyc', 'yyc', 'xyf', 'yyf', 
                           'frame_timestamp'])
DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp


### Create an array

Based on the previous analysis, we have chosen a time interval of 0.25 seconds to divide the data, ensuring sufficient observations for each colour.

In [4]:
# Create an array with steps of 0,25 s until the max timestemp in DatRaw is reached
Seconds_025=np.arange(0,int(np.max(DatRaw['frame_timestamp'])),0.25)
Seconds_025

array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 6.0225e+02, 6.0250e+02,
       6.0275e+02])

### Fill dataframe with values from DatRaw

In [5]:
# Map the color codes to their prefixes
color_map = {
    'r': 'red',
    'b': 'blue',
    'y': 'yellow'
}

for sec in Seconds_025:  # Loop through each interval
    # Filter data for the current time range
    dd = DatRaw[(DatRaw['frame_timestamp'] >= sec) & (DatRaw['frame_timestamp'] < sec + 0.25)]

    # Initialize a row for the current second
    row = {'frame_timestamp': sec}

    # Process each color
    for color_code, color_name in color_map.items():
        # Extract the data for center and front dots for the current color
        datcenter = dd[['x', 'y']][dd['name'].str.contains(f'{color_name}_center')]
        datfront = dd[['x', 'y']][dd['name'].str.contains(f'{color_name}_front')]

        if len(datcenter) > 0 and len(datfront) > 0:  # If both contain rows
            # Calculate the mean x and y coordinates for center and front dots
            row[f'x{color_code}c'] = np.mean(datcenter['x'])
            row[f'y{color_code}c'] = np.mean(datcenter['y'])
            row[f'x{color_code}f'] = np.mean(datfront['x'])
            row[f'y{color_code}f'] = np.mean(datfront['y'])

    # Append the row to the DataFrame
    DF = pd.concat([DF, pd.DataFrame([row])], ignore_index=True)

# Save to file
DF.to_csv("/Users/ruzenkakaldenbach/Desktop/filled.csv", index=False)

# Display the final DataFrame
DF

/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_61200/2425425246.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  DF = pd.concat([DF, pd.DataFrame([row])], ignore_index=True)


,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp
0,765.316377,879.993433,762.653500,874.610987,734.268108,493.786558,748.420308,519.338142,1006.295482,787.820078,986.268965,749.126704,0.00
1,764.100591,879.277665,761.693145,874.721471,718.035133,471.067141,740.246768,484.984101,1006.017803,787.794131,986.596370,748.670507,0.25
2,766.501134,878.722556,764.593577,873.058363,691.586754,454.706430,681.340206,457.224379,1005.666424,787.818096,986.538019,748.741710,0.50
3,767.319837,922.659777,767.101961,870.595491,656.939589,436.178309,640.702641,448.517544,1005.728230,788.595226,986.913178,749.325872,0.75
4,766.530858,928.530055,765.238423,868.136665,621.288114,433.016895,612.032934,454.397222,1005.873186,789.263442,986.784940,750.421330,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1056.682665,249.933598,1054.004560,281.906504,561.379615,511.242302,589.079484,533.459863,1143.638777,872.334399,1136.543666,812.269897,601.75
2408,1054.582125,251.487495,1053.509047,280.483679,559.796448,476.550640,590.675409,517.314159,1142.013003,881.322899,1138.363583,821.332907,602.00
2409,1054.929555,251.413135,1053.551428,279.326489,NaN,NaN,NaN,NaN,1144.370512,877.821007,1139.593426,818.445080,602.25
2410,1054.117098,248.248346,1053.513011,275.697791,NaN,NaN,NaN,NaN,1168.313839,875.107632,1158.615568,816.086531,602.50


### Reliability check

Compare number of means for each colour with old approach in Means_all_Intervals_all_Files:    
for DYAD06NF_ and	0.25s it should be   
blue 1911   
red  2124   
yellow  2290

In [6]:
# Iterate over colors and calculate the shape for each
for col in color_map.keys():  # Use the keys from color_map to iterate over 'r', 'b', 'y'
    # Filter rows for the current color
    color_df = DF[[f'x{col}c', f'y{col}c', f'x{col}f', f'y{col}f']].dropna(how='all')
    # Print the color and the shape of its subset
    print(color_map[col], color_df.shape)

red (2124, 4)
blue (1911, 4)
yellow (2290, 4)


### Interpolation

Replace missing values based on the value above and below the missing value within the same column.

In [7]:
from scipy.interpolate import interp1d

# Define a function to interpolate a column based on the row index (given the fix intervals between rows)
def interpolate_column_by_index(df, column_name):
    valid_idx = df.index[~df[column_name].isna()]  # Select row indices with valid (non-NaN) values
    valid_values = df[column_name][~df[column_name].isna()]  # Select the valid (non-NaN) values
    
    f = interp1d(valid_idx, valid_values, kind='linear', fill_value='extrapolate')
    df[column_name] = f(df.index)  # Interpolate for all row indices for the current column
    return df

# Apply interpolation for all coordinate columns
coordinate_columns = [col for col in DF.columns if col not in ['frame_timestamp']]
for column in coordinate_columns:
    DF = interpolate_column_by_index(DF, column)

# Save the interpolated DataFrame
DF.to_csv("/Users/ruzenkakaldenbach/Desktop/interpolated_by_index.csv", index=False)
DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp
0,765.316377,879.993433,762.653500,874.610987,734.268108,493.786558,748.420308,519.338142,1006.295482,787.820078,986.268965,749.126704,0.00
1,764.100591,879.277665,761.693145,874.721471,718.035133,471.067141,740.246768,484.984101,1006.017803,787.794131,986.596370,748.670507,0.25
2,766.501134,878.722556,764.593577,873.058363,691.586754,454.706430,681.340206,457.224379,1005.666424,787.818096,986.538019,748.741710,0.50
3,767.319837,922.659777,767.101961,870.595491,656.939589,436.178309,640.702641,448.517544,1005.728230,788.595226,986.913178,749.325872,0.75
4,766.530858,928.530055,765.238423,868.136665,621.288114,433.016895,612.032934,454.397222,1005.873186,789.263442,986.784940,750.421330,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1056.682665,249.933598,1054.004560,281.906504,561.379615,511.242302,589.079484,533.459863,1143.638777,872.334399,1136.543666,812.269897,601.75
2408,1054.582125,251.487495,1053.509047,280.483679,559.796448,476.550640,590.675409,517.314159,1142.013003,881.322899,1138.363583,821.332907,602.00
2409,1054.929555,251.413135,1053.551428,279.326489,558.213282,441.858978,592.271334,501.168455,1144.370512,877.821007,1139.593426,818.445080,602.25
2410,1054.117098,248.248346,1053.513011,275.697791,556.630116,407.167316,593.867260,485.022752,1168.313839,875.107632,1158.615568,816.086531,602.50


### Add columns for distances, facing/backing and angles